In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from textblob import Word, TextBlob
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from nltk.sentiment import SentimentIntensityAnalyzer
from warnings import filterwarnings

filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width', 200)
pd.set_option("display.max_colwidth", -1)

### 1. DATA PREPROCESSING

In [ ]:
train=pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding='latin1')
test=pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_test.csv",encoding='latin1')
df = test.append(train).reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
df_ = df[["OriginalTweet", "Sentiment"]]
df_.head()

#### 1.1 Text Preprocesing

Lower - upper case

In [ ]:
df_["OriginalTweet"]=df_["OriginalTweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
df_.head()

Dropping punctuation marks

In [ ]:
df_["OriginalTweet"]=df_["OriginalTweet"].str.replace('[^\w\s]', '')
df_.head()

Dropping numbers

In [ ]:
df_["OriginalTweet"]=df_["OriginalTweet"].str.replace('\d', '')
df_.head()

Stopwords

In [ ]:
sw=stopwords.words('english')
df_["OriginalTweet"]=df_["OriginalTweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
df_.head()

Rare Words

In [ ]:
dropping=pd.Series(' '.join(df_['OriginalTweet']).split()).value_counts()[-1000:]
df_["OriginalTweet"]=df_["OriginalTweet"].apply(lambda x: " ".join(x for x in x.split() if x not in dropping))

Lemmatization

In [ ]:
df_["OriginalTweet"]=df_["OriginalTweet"].apply(lambda x:" ".join([Word(word).lemmatize() for word in x.split()]))
df_.head()

### 2. DATA VISUALIZATION

Word Cloud

In [ ]:
text = " ".join(i for i in df_.OriginalTweet)
wordcloud = WordCloud(max_font_size=50,
                      max_words=300,
                      background_color="white",
                     width=1600, height=800).generate(text)
plt.figure(figsize=(20,10),facecolor='k')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

### 3. SENTIMENT ANALYSIS

In [ ]:
sia = SentimentIntensityAnalyzer()
df_["sentiment_label"] = df_["OriginalTweet"].apply(lambda x: "pos" if sia.polarity_scores(x)["compound"] > 0 else "neg")
df_.head(15)

### FEATURE ENGINEERING

* Count Vectors
* TF - IDF Vectors (words, characters, n-grams)
* Word Embeddings

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(df_["OriginalTweet"],
                                                    df_["sentiment_label"],
                                                    random_state=17)

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

### Count Vectors

In [ ]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)
x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

### TF-IDF Word Level

In [ ]:
tf_idf_word_vectorizer = TfidfVectorizer().fit(train_x)
x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

### TF-IDF N-Gram Level

In [ ]:
tf_idf_ngram_vectorizer = TfidfVectorizer(ngram_range=(2, 3)).fit(train_x)
x_train_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(train_x)
x_test_tf_idf_ngram = tf_idf_ngram_vectorizer.transform(test_x)

### TF-IDF Characters Level

In [ ]:
tf_idf_chars_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 3)).fit(train_x)
x_train_tf_idf_chars = tf_idf_chars_vectorizer.transform(train_x)
x_test_tf_idf_chars = tf_idf_chars_vectorizer.transform(test_x)

# RandomForestClassifier

In [ ]:
# TF-IDF Word-Level
rf_model = RandomForestClassifier().fit(x_train_tf_idf_word, train_y)
cross_val_score(rf_model, x_test_tf_idf_word, test_y, cv=5, n_jobs=-1).mean()

In [ ]:
# TF-IDF N-GRAM
rf_model = RandomForestClassifier().fit(x_train_tf_idf_ngram, train_y)
cross_val_score(rf_model, x_test_tf_idf_ngram, test_y, cv=5, n_jobs=-1).mean()

In [ ]:
# TF-IDF CHARLEVEL
rf_model = RandomForestClassifier().fit(x_train_tf_idf_chars, train_y)
cross_val_score(rf_model, x_test_tf_idf_chars, test_y, cv=5, n_jobs=-1).mean()

In [ ]:
# Count Vectors
rf_model = RandomForestClassifier().fit(x_train_count, train_y)
cross_val_score(rf_model, x_test_count, test_y, cv=5).mean()

### Example

In [ ]:
new_comment = pd.Series("I really need toilet paper")
new_comment = CountVectorizer().fit(train_x).transform(new_comment)

rf_model.predict(new_comment)

In [ ]:
new_comment = pd.Series("Good")
new_comment = CountVectorizer().fit(train_x).transform(new_comment)

rf_model.predict(new_comment)